# Plots

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import ttest_ind

pd.options.mode.chained_assignment = None

In [2]:
df = pd.read_json('experiments/results/nasbench201/cifar10/tier2high/averages.json').dropna(axis=1, how='all')
df.columns = pd.MultiIndex.from_tuples(df.columns.map(eval)).rename(['algorithm', 'seed'], level=[0, 1])
df.index.rename('epoch', inplace=True)
df = df.stack([0, 1]).groupby(['epoch', 'algorithm']).mean().unstack(1).drop(('estm', 're'), axis=1)
df.true.re[50:50 + df.true.re.dropna().shape[0]] = df.true.re.dropna()
df.true.re[:50] = np.nan
t2h = df

df = pd.read_json('experiments/results/nasbench201/cifar10/default/averages.json').dropna(axis=1, how='all')
df.columns = pd.MultiIndex.from_tuples(df.columns.map(eval)).rename(['algorithm', 'seed'], level=[0, 1])
df.index.rename('epoch', inplace=True)
df = df.stack([0, 1]).groupby(['epoch', 'algorithm']).mean().unstack(1).drop(('estm', 're'), axis=1)
df.true.re[50:50 + df.true.re.dropna().shape[0]] = df.true.re.dropna()
df.true.re[:50] = np.nan
dflt = df

In [3]:
t2h.true.mtnas.mean()

88.24615332225912

In [4]:
dflt.true.mtnas.mean()

88.21472592469546

In [5]:
ttest_ind(t2h.true.mtnas, dflt.true.mtnas)

Ttest_indResult(statistic=0.25740103551671756, pvalue=0.7969574542314518)